In [0]:
# Instalação das bibliotecas de Orquestração (LangChain/Graph), Conectores (Databricks) e Pesquisa (Tavily).
%pip install -U langchain langchain-community langchain-core langgraph databricks-langchain tavily-python matplotlib pandas mlflow
dbutils.library.restartPython()

  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.1
    Uninstalling langchain-1.2.1:
      Successfully uninstalled langchain-1.2.1
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Importações

# Bibliotecas padrão
import os
import json
from datetime import datetime, timedelta, timezone

# Dados e visualização
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# LangChain e LLMs
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
from databricks_langchain import ChatDatabricks

# Serviços externos e ML
from tavily import TavilyClient
import mlflow

# Configuração de Log
def log(msg: str, level: str = "INFO"):
    """
    Logger padronizado para rastreabilidade de execução.
    Formato: [YYYY-MM-DD HH:MM:SS] [LEVEL] Icon Mensagem
    """
    # Configuração de Fuso Horário (BRT)
    fuso_br = timezone(timedelta(hours=-3))
    timestamp = datetime.now(fuso_br).strftime("%Y-%m-%d %H:%M:%S")
    
    # Ícones visuais para facilitar leitura rápida dos logs
    icons = {
        "INFO": "ℹ️", 
        "WARN": "⚠️", 
        "ERROR": "❌", 
        "SUCCESS": "✅", 
        "SYSTEM": "⚙️",
        "TOOL": "🛠️",
        "AI": "🤖"
    }
    icon = icons.get(level, "")
    
    print(f"[{timestamp}] [{level}] {icon} {msg}")

log("Bibliotecas importadas e Logger configurado com sucesso.", "SYSTEM")

[2026-01-07 20:07:58] [SYSTEM] ⚙️ Bibliotecas importadas e Logger configurado com sucesso.


In [0]:
# Define o experimento

username = spark.sql("SELECT current_user()").collect()[0][0]
experiment_path = f"/Users/{username}/srag_agent_monitoring"
mlflow.set_experiment(experiment_path)

# Habilita o rastreamento automático para LangChain (capturar inputs, outputs, traces)
mlflow.langchain.autolog()

log(f"MLOps Ativado. Experiment Path: {experiment_path}", "SUCCESS")

[2026-01-07 20:08:15] [SUCCESS] ✅ MLOps Ativado. Experiment Path: /Users/eduardobdel@gmail.com/srag_agent_monitoring


In [0]:
# Arquitetura dos dados

CATALOG = "srag_prod" 
SCHEMA = "gold"       
VOLUME_NAME = "volume_imagens" # Storage para arquivos não estruturados (png)

# Garante a existência do volume para persistência dos gráficos gerados pelo agente
spark = SparkSession.builder.getOrCreate()
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME_NAME}")

VOLUME_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME_NAME}"
TABLE_DAILY = f"{CATALOG}.{SCHEMA}.gold_srag_daily"
TABLE_MONTHLY = f"{CATALOG}.{SCHEMA}.gold_srag_monthly"

log(f"Ambiente de Dados Configurado.", "SYSTEM")
log(f"Caminho do Volume: {VOLUME_PATH}", "INFO")
log(f"Tabelas Mapeadas: {TABLE_DAILY}, {TABLE_MONTHLY}", "INFO")

[2026-01-07 20:08:15] [SYSTEM] ⚙️ Ambiente de Dados Configurado.
[2026-01-07 20:08:15] [INFO] ℹ️ Caminho do Volume: /Volumes/srag_prod/gold/volume_imagens
[2026-01-07 20:08:15] [INFO] ℹ️ Tabelas Mapeadas: srag_prod.gold.gold_srag_daily, srag_prod.gold.gold_srag_monthly


In [0]:
# Validação dos dados

def run_quality_gate():
    """
    Valida os dados da tabela Gold. Se falhar, interrompe o notebook.
    Regras:
    1. Apenas anos 2024 e 2025.
    2. Sem casos negativos.
    3. Sem datas nulas.
    """
    log("Executando Validação de Qualidade de Dados (Data Quality Gate)...", "SYSTEM")
    
    spark = SparkSession.builder.getOrCreate()
    
    # A função 'sum' conta quantas linhas violam cada regra
    query_check = f"""
        SELECT 
            COUNT(*) as total_linhas,
            SUM(CASE WHEN year(data_referencia) NOT IN (2024, 2025) THEN 1 ELSE 0 END) as erro_anos,
            SUM(CASE WHEN total_casos < 0 THEN 1 ELSE 0 END) as erro_negativos,
            SUM(CASE WHEN data_referencia IS NULL THEN 1 ELSE 0 END) as erro_nulos
        FROM {TABLE_DAILY}
    """
    
    # Coleta o resultado
    check = spark.sql(query_check).collect()[0]
    
    erros = []
    
    # 1. Validação de Anos (2024/2025 apenas)
    if check['erro_anos'] > 0:
        erros.append(f"Regra de Ano: {check['erro_anos']} registros fora de 2024/2025.")
        
    # 2. Validação de Negativos
    if check['erro_negativos'] > 0:
        erros.append(f"Regra de Negativos: {check['erro_negativos']} registros com casos negativos.")
        
    # 3. Validação de Nulos
    if check['erro_nulos'] > 0:
        erros.append(f"Regra de Nulos: {check['erro_nulos']} registros com data vazia.")
        
    # Verifica se houve algum erro
    if erros:
        msg_erro = "\n".join(erros)
        log(f"FALHA CRÍTICA DE DATA QUALITY:\n{msg_erro}", "ERROR")
        raise ValueError(f"🚨 O notebook foi interrompido para segurança dos dados.")
    
    log(f"Dados Aprovados: {check['total_linhas']} registros validados (2024-2025, Sem Nulos, Sem Negativos).", "SUCCESS")

# Roda a validação
run_quality_gate()

[2026-01-07 20:08:16] [SYSTEM] ⚙️ Executando Validação de Qualidade de Dados (Data Quality Gate)...
[2026-01-07 20:08:16] [SUCCESS] ✅ Dados Aprovados: 721 registros validados (2024-2025, Sem Nulos, Sem Negativos).


In [0]:
# Query e Teste Unitário

@tool
def get_latest_srag_metrics() -> str:
    """
    Consulta o banco de dados para obter as métricas mais recentes de SRAG.
    Retorna: Um JSON com total de casos, e taxas de mortalidade, UTI e vacinação.
    """
    try:
        # log("Consultando métricas no Lakehouse...", "TOOL") # Opcional: Descomentar se quiser muito detalhe
        spark = SparkSession.builder.getOrCreate()
        
        query = f"""
        SELECT 
            cast(data_referencia as string) as data_referencia,
            cast(total_casos as int) as total_casos,
            cast(taxa_aumento_casos_perc as double) as taxa_aumento_casos_perc,
            cast(taxa_mortalidade_perc as double) as taxa_mortalidade_perc,
            cast(taxa_ocupacao_uti_perc as double) as taxa_ocupacao_uti_perc,
            cast(taxa_vacinacao_pacientes_perc as double) as taxa_vacinacao_pacientes_perc
        FROM {TABLE_DAILY}
        ORDER BY data_referencia DESC
        LIMIT 1
        """
        
        df = spark.sql(query).toPandas()
        
        if df.empty:
            log("ALERTA: Tabela vazia ao buscar métricas.", "WARN")
            return "ALERTA: A tabela SQL retornou vazio. Verifique se o pipeline rodou."
            
        result = df.iloc[0].to_dict()

        # Enriquecimento semântico 
        result["analysis_scope"] = "short_term"
        result["comparison_window"] = "daily_vs_previous"
        result["data_nature"] = "preliminary"

        return json.dumps(result, ensure_ascii=False)

    except Exception as e:
        log(f"Erro na Tool de Métricas: {str(e)}", "ERROR")
        return f"ERRO CRÍTICO NO SPARK/SQL: {str(e)}"

# Teste manual imediato
log("Teste Unitário - Métricas:", "SYSTEM")
print(get_latest_srag_metrics.invoke({}))

[2026-01-07 20:08:17] [SYSTEM] ⚙️ Teste Unitário - Métricas:
{"data_referencia": "2025-12-21", "total_casos": 4, "taxa_aumento_casos_perc": -33.33, "taxa_mortalidade_perc": 0.0, "taxa_ocupacao_uti_perc": 25.0, "taxa_vacinacao_pacientes_perc": 25.0, "analysis_scope": "short_term", "comparison_window": "daily_vs_previous", "data_nature": "preliminary"}


Trace(trace_id=tr-d08378236bdf8582eafb92d660f56ef3)

In [0]:
# Visualização

@tool
def generate_srag_charts() -> str:
    """
    Gera gráficos de linha (30 dias) e barras (12 meses) sobre SRAG.
    Padrão de Data:
    - Diário: dd/mm (Ex: 28/12)
    - Mensal: mm/aaaa (Ex: 12/2024) - Ano com 4 dígitos para evitar confusão com dia.
    """
    try:
        log("Iniciando geração de gráficos...", "TOOL")
        
        # GRÁFICO 1: DIÁRIO (30 DIAS) - Linha
        query_daily = f"SELECT data_referencia, total_casos FROM {TABLE_DAILY} ORDER BY data_referencia DESC LIMIT 30"
        df_daily = spark.sql(query_daily).toPandas().sort_values('data_referencia')
        
        path_daily = "Sem dados diários"
        if not df_daily.empty:
            df_daily['data_referencia'] = pd.to_datetime(df_daily['data_referencia'])
            
            plt.figure(figsize=(10, 5))
            plt.plot(df_daily['data_referencia'], df_daily['total_casos'], marker='o', color='#1f77b4', linewidth=2)
            
            # Formatação Eixo X (Diário) -> dd/mm
            ax1 = plt.gca()
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))
            ax1.xaxis.set_major_locator(mdates.DayLocator(interval=2)) 
            
            plt.title('Casos SRAG - Últimos 30 Dias')
            plt.ylabel('Casos')
            plt.grid(True, linestyle='--', alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            
            path_daily = f"{VOLUME_PATH}/grafico_diario.png"
            plt.savefig(path_daily)
            plt.close()

        # GRÁFICO 2: MENSAL (12 MESES) - Barras
        query_monthly = f"SELECT mes_referencia, total_casos FROM {TABLE_MONTHLY} ORDER BY mes_referencia DESC LIMIT 12"
        df_monthly = spark.sql(query_monthly).toPandas().sort_values('mes_referencia')
        
        path_monthly = "Sem dados mensais"
        if not df_monthly.empty:
            df_monthly['mes_referencia'] = pd.to_datetime(df_monthly['mes_referencia'])
            
            plt.figure(figsize=(10, 5))
            plt.bar(df_monthly['mes_referencia'], df_monthly['total_casos'], color='#ff7f0e', width=20)
            
            # Formatação Eixo X (Mensal) -> mm/aaaa (Ex: 01/2025)
            ax2 = plt.gca()
            ax2.xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y')) 
            ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
            
            plt.title('Casos SRAG - Últimos 12 Meses')
            plt.ylabel('Casos')
            plt.grid(True, axis='y', linestyle='--', alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            
            path_monthly = f"{VOLUME_PATH}/grafico_mensal.png"
            plt.savefig(path_monthly)
            plt.close()

        log(f"Gráficos persistidos no Volume: {path_daily}, {path_monthly}", "SUCCESS")
        return f"Gráficos atualizados salvos em:\n1. {path_daily}\n2. {path_monthly}"

    except Exception as e:
        log(f"Falha ao gerar gráficos: {e}", "ERROR")
        return f"ERRO AO GERAR GRÁFICOS: {str(e)}"

In [0]:
# Contexto dados

try:
    token = dbutils.secrets.get(scope="my_srag_scope", key="tavily_api_key")
    os.environ["TAVILY_API_KEY"] = token
    log("API Key do Tavily carregada via Secrets.", "SUCCESS")
except Exception as e:
    log(f"Erro ao carregar a API Key: {e}", "ERROR")

@tool
def get_epidemiological_context() -> str:
    """
    Realiza 'Grounding' (Ancoragem) do modelo em dados externos em tempo real.
    Estratégia:
    1. Busca Macro (Global/OMS) para identificar novas variantes.
    2. Busca Micro (Brasil/Fiocruz) para dados epidemiológicos locais.
    Isso reduz a chance do modelo inventar contextos.
    """
    try:
        client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
        
        # Passo 1: Contexto Internacional
        log("Investigando cenário Global (OMS/CDC)...", "TOOL")
        response_global = client.search(
            query="Global respiratory virus trends WHO CDC influenza covid epidemiological update", 
            search_depth="basic", topic="news", max_results=2, include_answer=True
        )
        
        # Passo 2: Contexto Nacional
        log("Investigando cenário Brasil (InfoGripe/Fiocruz)...", "TOOL")
        response_br = client.search(
            query="Boletim InfoGripe Fiocruz Brasil cenário atual SRAG covid influenza", 
            search_depth="basic", topic="news", max_results=3, include_answer=True
        )
        
        # Montagem do Contexto
        contexto = f"""
        | RELATÓRIO DE INTELIGÊNCIA EXTERNA |
        1. GLOBAL: {response_global.get('answer', 'N/A')}
        2. NACIONAL: {response_br.get('answer', 'N/A')}
        
        FONTES NACIONAIS:
        """
        for res in response_br.get('results', []):
            contexto += f"- {res['title']}: {res['content'][:200]}...\n"
            
        return contexto

    except Exception as e:
        log(f"Erro na busca externa: {str(e)}", "ERROR")
        return f"Erro na busca externa: {str(e)}"

[2026-01-07 20:08:18] [SUCCESS] ✅ API Key do Tavily carregada via Secrets.


In [0]:
# Definição do Agente

# 1. Configuração do Modelo: O parâmetro 'model' define qual endpoint de inferência será acionado.
llm = ChatDatabricks(model="databricks-meta-llama-3-3-70b-instruct")

# 2. Binding das Ferramentas
# O Agente precisa de uma lista de 'tools' disponíveis para saber o que ele pode fazer.
tools = [get_latest_srag_metrics, generate_srag_charts, get_epidemiological_context]

# 3. Criação do Executor (runtime que pega o pensamento do LLM e efetivamente roda as ferramentas Python).
try:
    agent_executor = create_agent(llm, tools)
    log(f"Agente configurado e pronto! Modelo: {llm.model}", "SUCCESS")
except Exception as e:
    log(f"Erro ao criar agente: {e}", "ERROR")

[2026-01-07 20:08:19] [SUCCESS] ✅ Agente configurado e pronto! Modelo: databricks-meta-llama-3-3-70b-instruct


In [0]:
# Execução final

from langchain_core.messages import SystemMessage
from datetime import datetime, timedelta, timezone

# 1. Configuração de Data/Hora
fuso_br = timezone(timedelta(hours=-3))
data_hora = datetime.now(fuso_br).strftime("%d/%m/%Y às %H:%M")

# 2. Definição do System Prompt
system_instructions = """
ATENÇÃO: Você deve seguir este roteiro de forma estrita.

ORDEM DE EXECUÇÃO
1. Chame as ferramentas internas:
   - get_latest_srag_metrics
   - generate_srag_charts
2. Chame a ferramenta externa:
   - get_epidemiological_context
   Aguarde o retorno e use apenas o texto recebido.

REGRAS DE LINGUAGEM
- O campo taxa_aumento_casos_perc representa variação percentual.
- Valores negativos indicam queda.
- É proibido usar expressões como:
  "aumento negativo", "taxa de aumento negativa" ou "aumento de -X%".
- Use apenas:
  "redução de X%", "queda de X%" ou "os casos diminuíram X%".

REGRAS EPIDEMIOLÓGICAS
- Comparações entre períodos recentes (diário ou último período disponível)
  devem ser descritas apenas como:
  "sinal recente", "variação pontual" ou "movimento de curto prazo".
- Não use a palavra "tendência" para análises de curto prazo.
- Não faça inferências causais ou preditivas.
- Se o número de casos for baixo ou o período for recente:
  - Use linguagem condicional.
  - Declare que os dados são preliminares.
  - Cite possível atraso de notificação (apenas uma vez).

REGRA ABSOLUTA SOBRE MORTALIDADE
- Se o número de óbitos for igual a zero:
  - NÃO use os termos "taxa de mortalidade" ou "0%".
  - Use exclusivamente a frase:
    "Não houve registro de óbitos no período analisado."
- Não complemente.
- Não explique.
- Não faça conversões numéricas.
- Qualquer violação invalida a resposta.

ESTRUTURA DO RELATÓRIO

Análise dos Dados Internos (DataSUS)
- Comece pelos valores observados:
  total de casos, UTI, vacinação e óbitos.
- Em seguida, descreva a variação percentual (queda ou aumento).
- Finalize com uma única frase de cautela
  (curto prazo, dados preliminares, atraso de notificação).
- Se não houver óbitos, use apenas a frase obrigatória.

Panorama Global
- Resuma apenas o conteúdo retornado pela ferramenta externa.
- Não extrapole nem interprete além do texto recebido.

Cenário Brasil (InfoGripe/Fiocruz)
- Resuma apenas o conteúdo retornado pela ferramenta externa.
- Se não houver informação, informe que a ferramenta não retornou dados.

Conclusão Técnica
- Texto curto, objetivo e cauteloso.
- Não use "tendência" se baseado em curto prazo.
- Evite afirmações absolutas.
"""

log(f"Iniciando ciclo de análise do Agente...", "SYSTEM")

# 3. Montagem do Payload
inputs = {
    "messages": [
        SystemMessage(content=system_instructions),
        # Aqui entra o PROMPT DO USUÁRIO
        ("user", "Gere o relatório de monitoramento SRAG de hoje.")
    ]
}

# 4. Execução com Logs Detalhados
try:
    for chunk in agent_executor.stream(inputs, stream_mode="values", config={"recursion_limit": 25}):
        message = chunk["messages"][-1]
        
        if message.type == "ai" and not message.tool_calls:
            # Mostra o pensamento ou a resposta final
            print(f"\n--- [AGENTE PENSANDO / RESPOSTA] ---\n{message.content}")
        elif message.type == "tool":
            log(f"Ferramenta acionada: {message.name}", "TOOL")
    
    hora_fim = datetime.now(fuso_br).strftime("%H:%M:%S")
    print("\n" + ("=" * 50))
    log(f"Processo finalizado às {hora_fim}", "SUCCESS")

except Exception as e:
    log(f"Erro na execução do loop do agente: {e}", "ERROR")

[2026-01-07 20:22:55] [SYSTEM] ⚙️ Iniciando ciclo de análise do Agente...
[2026-01-07 20:22:56] [TOOL] 🛠️ Ferramenta acionada: get_latest_srag_metrics
[2026-01-07 20:22:56] [TOOL] 🛠️ Iniciando geração de gráficos...
[2026-01-07 20:22:58] [SUCCESS] ✅ Gráficos persistidos no Volume: /Volumes/srag_prod/gold/volume_imagens/grafico_diario.png, /Volumes/srag_prod/gold/volume_imagens/grafico_mensal.png
[2026-01-07 20:22:58] [TOOL] 🛠️ Ferramenta acionada: generate_srag_charts
[2026-01-07 20:22:58] [TOOL] 🛠️ Investigando cenário Global (OMS/CDC)...
[2026-01-07 20:22:58] [TOOL] 🛠️ Investigando cenário Brasil (InfoGripe/Fiocruz)...
[2026-01-07 20:22:59] [TOOL] 🛠️ Ferramenta acionada: get_epidemiological_context

--- [AGENTE PENSANDO / RESPOSTA] ---
Análise dos Dados Internos (DataSUS)

De acordo com os dados mais recentes, o total de casos de SRAG é de 4, com uma taxa de aumento de casos de -33,33%, indicando uma redução de 33,33% nos casos. A taxa de mortalidade é de 0,0%, e não houve registro d

Trace(trace_id=tr-bae47d7eb14b7887b168340c81646ab0)

In [0]:
# Registro de Execução

# Salvar o resultado do agente
with mlflow.start_run(run_name="Relatorio Diario SRAG") as run:
    log("Iniciando registro de auditoria no MLflow...", "INFO")
    
    # 1. Logamos os Parâmetros
    mlflow.log_param("data_execucao", data_hora)
    mlflow.log_param("modelo_usado", "llama-3-70b")
    
    # 2. Log do Texto Final
    nome_arquivo_relatorio = "relatorio_srag.md"
    
    # Pega a última mensagem válida do loop anterior
    # (Nota: Assume que a variável 'message' ainda está na memória da célula anterior)
    try:
        texto_final = message.content 
        mlflow.log_text(texto_final, nome_arquivo_relatorio)
        log(f"Texto do relatório salvo: {nome_arquivo_relatorio}", "INFO")
    except NameError:
        log("Variável 'message' não encontrada. O agente rodou?", "WARN")
    
    # 3. Logamos os Gráficos. O MLflow copia as imagens do Volume para dentro do Experimento
    try:
        mlflow.log_artifact(f"{VOLUME_PATH}/grafico_diario.png", artifact_path="graficos")
        mlflow.log_artifact(f"{VOLUME_PATH}/grafico_mensal.png", artifact_path="graficos")
        log("Artefatos visuais (Gráficos) anexados ao experimento.", "SUCCESS")
    except Exception as e:
        log(f"Não foi possível logar as imagens: {e}", "WARN")

    print("-" * 100)
    log("Sucesso! Execução auditada no MLflow.", "SUCCESS")
    log(f"Link para Experiment: {experiment_path}", "INFO")

[2026-01-07 20:08:30] [INFO] ℹ️ Iniciando registro de auditoria no MLflow...
[2026-01-07 20:08:31] [INFO] ℹ️ Texto do relatório salvo: relatorio_srag.md
[2026-01-07 20:08:32] [SUCCESS] ✅ Artefatos visuais (Gráficos) anexados ao experimento.
----------------------------------------------------------------------------------------------------
[2026-01-07 20:08:32] [SUCCESS] ✅ Sucesso! Execução auditada no MLflow.
[2026-01-07 20:08:32] [INFO] ℹ️ Link para Experiment: /Users/eduardobdel@gmail.com/srag_agent_monitoring
